In [32]:
import os,glob
import nilearn.glm
from nilearn.interfaces.fmriprep import load_confounds
import pandas as pd
import numpy as np
import nilearn.plotting
from nilearn.glm.first_level import FirstLevelModel

In [79]:
CONTRASTS = {
    "painMainFx": np.asarray([1,1,1,1,1,0,0,0]),
    "painRegression": np.asarray([1,2,3,2,2,0,0,0])/10.,
    "painHighMinusLow": np.asarray([-1,0,1,0,0,0,0,0]),
    "cueHighMinusLow": np.asarray([0,0,0,0,0,-1,0,1]),
    "hypoalgesia": np.asarray([0,-1,0,1,0,0,0,0]),
    "hyperalgesia": np.asarray([0,-1,0,0,1,0,0,0]),
}

def compute_contrasts_run(bold_file, bold_file_tedana, mask_file, evt_file, smoothing_fwhm=5, t_r=1.48):
    evts = pd.read_csv(evt_file, delimiter='\t')
    cfnds = load_confounds(bold_file, motion="derivatives")[0]
    fmri_glm = FirstLevelModel(
        t_r=t_r,
        mask_img=mask_file,
        drift_model=None,
        smoothing_fwhm=smoothing_fwhm,
    )
    fmri_glm.fit(bold_file_tedana, events=evts, confounds=cfnds)
    return {ctx_name:fmri_glm.compute_contrast(ctx) for ctx_name, ctx in CONTRASTS.items()}




def compute_contrasts(bold_files, bold_files_tedana, mask_file, evt_files, smoothing_fwhm=5, t_r=1.48):
    evts = [pd.read_csv(evt_file, delimiter='\t') for evt_file in evt_files]
    cfnds = [load_confounds(bold_file, motion="derivatives")[0] for bold_file in bold_files]
    fmri_glm = FirstLevelModel(
        t_r=t_r,
        mask_img=mask_file,
        drift_model=None,
        smoothing_fwhm=smoothing_fwhm,
    )
    fmri_glm.fit(bold_files_tedana, events=evts, confounds=cfnds)
    return {ctx_name:fmri_glm.compute_contrast(ctx) for ctx_name, ctx in CONTRASTS.items()}


def save_contrasts(path, contrasts):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    for ctx_name, ctx in contrasts.items():
        ctx.to_filename(f"{path}_contrast-{ctx_name}_stat-effect_statmap.nii.gz")
        max_val = ctx.dataobj.max()
        view = nilearn.plotting.view_img(ctx, threshold=max(2,max_val*.4), vmax=max_val*.8)
        view.save_as_html(f"{path}_contrast-{ctx_name}_stat-effect_statmap.html")
        del view

In [88]:
contrasts=dict()
usable_runs = {
 #   2: [1,2,3],
    3: [1,2],
    4:[1,2,3],
}
for sub,ur in usable_runs.items():
    print(f"processing sub-0{sub}")
    ur = "".join(map(str,ur))
    contrasts[sub] = compute_contrasts(
        bold_files = sorted(glob.glob(f"/scratch/PDPain/pdpain.fmriprep/sub-{sub:02d}/func/sub-{sub:02d}_task-painexpectancy_run-0[{ur}]_part-mag_space-T1w_desc-preproc_bold.nii.gz")),
        bold_files_tedana = sorted(glob.glob(f"/scratch/PDPain/pdpain.tedana/sub-{sub:02d}/func/sub-{sub:02d}_task-painexpectancy_run-0[{ur}]/space-MNI152NLin2009cAsym_desc-denoised_bold.nii.gz")),
        mask_file = f"/scratch/PDPain/pdpain.fmriprep/sub-{sub:02d}/func/sub-{sub:02d}_task-painexpectancy_run-02_part-mag_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz",
        evt_files = sorted(glob.glob(f"/scratch/PDPain/pdpain.fmriprep/sourcedata/bids/sub-{sub:02d}/func/sub-{sub:02d}_task-painexpectancy_run-0[{ur}]_events.tsv")),
    )
    save_contrasts(f"./sub-{sub:02d}/stats/sub-{sub:02d}_task-painexpectancy_space-MNI152NLin2009cAsym", contrasts[sub])

processing sub-03


/home/bpinsard/.virtualenvs/neuromod_data3.10/lib/python3.10/site-packages/nilearn/glm/first_level/experimental_paradigm.py:167: UserWarning: The following unexpected columns in events data will be ignored: Unnamed: 0
  warnings.warn(
/home/bpinsard/.virtualenvs/neuromod_data3.10/lib/python3.10/site-packages/nilearn/glm/first_level/experimental_paradigm.py:167: UserWarning: The following unexpected columns in events data will be ignored: TODO -- fill in rows and add more tab-separated columns if desired, response_time, stim_file
  warnings.warn(
/tmp/ipykernel_1303321/4082526485.py:35: UserWarning: One contrast given, assuming it for all 2 runs
  return {ctx_name:fmri_glm.compute_contrast(ctx) for ctx_name, ctx in CONTRASTS.items()}
/home/bpinsard/.virtualenvs/neuromod_data3.10/lib/python3.10/site-packages/nilearn/glm/contrasts.py:108: UserWarning: t contrasts should be of length P=33, but it has length 8. The rest of the contrast was padded with zeros.
  reg = regression_result[label_

processing sub-04


/home/bpinsard/.virtualenvs/neuromod_data3.10/lib/python3.10/site-packages/nilearn/glm/first_level/experimental_paradigm.py:167: UserWarning: The following unexpected columns in events data will be ignored: Unnamed: 0
  warnings.warn(
/home/bpinsard/.virtualenvs/neuromod_data3.10/lib/python3.10/site-packages/nilearn/glm/first_level/experimental_paradigm.py:167: UserWarning: The following unexpected columns in events data will be ignored: Unnamed: 0
  warnings.warn(
/home/bpinsard/.virtualenvs/neuromod_data3.10/lib/python3.10/site-packages/nilearn/glm/first_level/experimental_paradigm.py:167: UserWarning: The following unexpected columns in events data will be ignored: Unnamed: 0
  warnings.warn(
/tmp/ipykernel_1303321/4082526485.py:35: UserWarning: One contrast given, assuming it for all 3 runs
  return {ctx_name:fmri_glm.compute_contrast(ctx) for ctx_name, ctx in CONTRASTS.items()}
/home/bpinsard/.virtualenvs/neuromod_data3.10/lib/python3.10/site-packages/nilearn/glm/contrasts.py:108: